In [1]:
# Get all the players from Sleeper and store the pertinent information in a SQLite database
import requests
import sqlite3
import json
from sleeper import get_all_players, get_users_in_league, get_all_rosters
from manage import clean_player_data
from config import LEAGUE_LIST
import pandas as pd
LEAGUE_ID = "1234567890123456789"
SLEEPER_API = "https://api.sleeper.app/v1/players/nfl"


In [19]:
# Get all the players from Sleeper
players_json = get_all_players()
players_df = clean_player_data(players_json)

# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
players_df.to_sql('players', conn, if_exists='replace', index=True)
conn.commit()
conn.close()

# Insert the league id with the league name into the db
This is a manual process of keeping this up to date

In [2]:
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Remove the coaches table if it exists already
conn.execute("DROP TABLE IF EXISTS league")

# Convert to a DataFrame
df = pd.DataFrame(list(LEAGUE_LIST.items()), columns=['name', 'league_id'])
df.to_sql('league', conn, if_exists='append', index=False)


7

# Insert the coaches into the database

In [3]:
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Remove the coaches table if it exists already
conn.execute("DROP TABLE IF EXISTS coaches")
# Get all the coaches in each of the leagues
for league in LEAGUE_LIST:
    coaches_df = get_users_in_league(LEAGUE_LIST[league])
    coaches_df.to_sql('coaches', conn, if_exists='append', index=False)

# Update the team names if they aren't already set by combining the display name and Team like "Team" + Display Name from the row
conn.execute("UPDATE coaches SET team_name = 'Team ' + display_name WHERE team_name IS NULL")

# Close the connection
conn.close()

# Now get the drafts from every league and put these in the database

In [4]:
# Get the drafts for each league
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
conn.execute("DROP TABLE IF EXISTS rosters")
for league in LEAGUE_LIST:
    rosters = get_all_rosters(LEAGUE_LIST[league])
    # Expand the players columns into individual rows keeping just the player_id, the owner_id, and league_id
    rosters_df = rosters.explode('players')[['players','owner_id','league_id']].rename(columns={'players':'player_id'})
    duplicates = rosters_df.duplicated()
    print(f"Number of duplicate rows: {duplicates.sum()}")
    
    rosters_df.to_sql('rosters', conn, if_exists='append', index=False)
# Close the connection
conn.close()


Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0


# Load Bye Weeks into the Database

In [ ]:
# Load bye weeks into the SQLite database
conn = sqlite3.connect('../data/league_db.db')
conn.execute("DROP TABLE IF EXISTS team")
# Open the file with the bye weeks
with open('../data/nfl_bye_weeks_2024.csv', 'r') as file:
    # Read the data into a dataframe
    bye_weeks = pd.read_csv(file)
    # Insert the data into the database
    bye_weeks.to_sql('team', conn, if_exists='replace', index=False)
conn.close()

# Create spreadsheet of rostered players

In [5]:
conn = sqlite3.connect('../data/league_db.db')
rows = conn.execute(f'Select p.full_name, p.college, p.years_exp, p.team, p.position, c.display_name, c.league_id, t."Bye Week"'
                    'FROM players p '
                    'JOIN rosters r on p.player_id = r.player_id '
                    'JOIN coaches c on r.owner_id = c.user_id and r.league_id = c.league_id '
                    'JOIN team t on p.team=t.team').fetchall()
# turn the rows into a dataframe
df = pd.DataFrame(rows, columns=['full_name','college', 'years_experience','team', 'position', 'coach','league_id','bye'])
conn.close()

# Create a reverse dictionary to map IDs to names
reverse_LEAGUE_LIST = {str(v): k for k, v in LEAGUE_LIST.items()}
# Ensure the league_id column is of type string
df['league_id'] = df['league_id'].astype(str)

# Now replace the league name with the league id
df['league_name'] = df['league_id'].map(reverse_LEAGUE_LIST)
df.drop(columns=['league_id'], inplace=True)

# Output the data to a csv file
df.to_csv('../data/players_rostered.csv', index=False)
df

,full_name,college,years_experience,team,position,coach,bye,league_name
0,Jonathan Mingo,Ole Miss,1.0,CAR,WR,ChiJV,11,MainLeague
1,Rashee Rice,SMU,1.0,KC,WR,ChiJV,6,MainLeague
2,Roschon Johnson,Texas,1.0,CHI,RB,ChiJV,7,MainLeague
3,Brenden Rice,USC,0.0,LAC,WR,ChiJV,5,MainLeague
4,Luke McCaffrey,Rice,0.0,WAS,WR,ChiJV,14,MainLeague
...,...,...,...,...,...,...,...,...
1405,Kenneth Walker,Michigan State,2.0,SEA,RB,WestCoastNonsense,10,Quickdraft 2
1406,Brock Purdy,Iowa State,2.0,SF,QB,WestCoastNonsense,9,Quickdraft 2
1407,Anthony Richardson,Florida,1.0,IND,QB,WestCoastNonsense,14,Quickdraft 2
1408,C.J. Stroud,Ohio State,1.0,HOU,QB,WestCoastNonsense,14,Quickdraft 2


In [48]:
# Run a sqlite query
def run_query(q):
    conn = sqlite3.connect('../data/league_db.db')
    rows = conn.execute(q).fetchall()
    df = pd.DataFrame(rows)
    return df

def get_column_names(table_name):
    # Connect to the SQLite3 database
    conn = sqlite3.connect('../data/league_db.db')
    cursor = conn.cursor()
    
    # Execute PRAGMA table_info to get the column information
    cursor.execute(f"PRAGMA table_info({table_name})")
    
    # Fetch all results
    columns_info = cursor.fetchall()
    
    # Extract column names
    column_names = [info[1] for info in columns_info]
    
    # Close the connection
    conn.close()
    
    return column_names

run_query('Select p.full_name, r.league_id, p.player_id '
                    'FROM players p join rosters r on r.player_id = p.player_id WHERE full_name like "%Davante Adams%"')


,0,1,2
0,Davante Adams,1060410179696533504,2133
1,Davante Adams,1097749730030931968,2133
2,Davante Adams,1103796139196858368,2133
3,Davante Adams,1105340566650339328,2133
4,Davante Adams,1104619976205471744,2133
5,Davante Adams,1104635110046986240,2133


In [49]:
run_query('Select * from rosters where player_id = 2133')

,0,1,2
0,2133,987074330058113024,1060410179696533504
1,2133,990442792889405440,1097749730030931968
2,2133,998692352560746496,1103796139196858368
3,2133,564369169909952512,1105340566650339328
4,2133,564369169909952512,1104619976205471744
5,2133,990442792889405440,1104635110046986240
